** Queue **

You have deque, you have stacks — Queue closes the loop on the classic linear data structures before we move into search and sort.

---

# Queue

**The one-liner:** A queue is a FIFO (First In, First Out) structure — the first item added is the first item removed. Think of a line at a grocery store.

**Mental model:** Opposite of a stack. Stack is LIFO (last plate on top gets taken first). Queue is FIFO (first person in line gets served first).

---
A deque (double-ended queue) is **not** strictly a queue — it's actually more powerful than a queue. Here's why:

---

**A strict queue has only ONE rule:**
```
Add to the BACK
Remove from the FRONT

BACK →  [3, 2, 1]  → FRONT
         in              out
```
That's it. No exceptions. This is called **FIFO — First In, First Out**. Like a line at a coffee shop — first person in line is first person served.

---

**A deque breaks that rule — it has FOUR doors:**
```
Add to BACK     ──►  [3, 2, 1]  ◄──  Add to FRONT
Remove from BACK ◄──  [3, 2, 1]  ──►  Remove from FRONT
```
You can push and pop from **either end**. That freedom is what makes it a deque, not a queue.

---

**So what makes a deque behave like a strict queue?**

You just agree to only use two of its four doors:

```python
from collections import deque

q = deque()

q.append(val)      # ONLY add to the back
q.popleft()        # ONLY remove from the front
```

Ignore `appendleft()` and `pop()` entirely. You've now got a strict queue — FIFO guaranteed — just built on top of a deque.

---

The deque is the **hardware**. The queue is the **rule you impose on it**.
## Three Ways to Implement in Python

**Option 1 — deque (the right way):**
```python
from collections import deque

queue = deque()
queue.append("a")      # enqueue → deque(['a'])
queue.append("b")      # enqueue → deque(['a', 'b'])
queue.popleft()        # dequeue → returns 'a'  (FIFO)
```

**Option 2 — list (the wrong way, but common):**
```python
queue = []
queue.append("a")      # enqueue — O(1)
queue.pop(0)           # dequeue — O(n) ← this is why deque exists
```

**Option 3 — queue.Queue (thread-safe, rarely needed on LeetCode):**
```python
from queue import Queue

q = Queue()
q.put("a")             # enqueue
q.get()                # dequeue → 'a'
```

For LeetCode: always use `deque`. For production threading: use `Queue`.

---

## Core Operations

```python
from collections import deque

q = deque()

# Enqueue (add to back)
q.append(1)
q.append(2)
q.append(3)    # deque([1, 2, 3])

# Peek (look at front without removing)
front = q[0]   # 1

# Dequeue (remove from front)
q.popleft()    # returns 1, deque([2, 3])

# Check empty
len(q) == 0    # False
not q          # False
```

---

## Complexity

| Operation | deque Queue |
|-----------|-------------|
| Enqueue (append) | O(1) |
| Dequeue (popleft) | O(1) |
| Peek front | O(1) |
| Search | O(n) |

---

## LeetCode Patterns

**BFS — the #1 queue use case:**
```python
from collections import deque

def bfs(graph, start):
    queue = deque([start])
    visited = set([start])
    order = []

    while queue:
        node = queue.popleft()
        order.append(node)

        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return order

graph = {
    "A": ["B", "C"],
    "B": ["D"],
    "C": ["D"],
    "D": []
}
print(bfs(graph, "A"))  # ['A', 'B', 'C', 'D']
```

**Level-order tree traversal:**
```python
def level_order(root):
    if not root:
        return []
    
    queue = deque([root])
    result = []

    while queue:
        level = []
        for _ in range(len(queue)):   # process one level at a time
            node = queue.popleft()
            level.append(node.val)
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        result.append(level)
    return result
```

---

## Stack vs Queue — The Mental Anchor

| | Stack | Queue |
|-|-------|-------|
| Order | LIFO | FIFO |
| Add | push (append) | enqueue (append) |
| Remove | pop (pop) | dequeue (popleft) |
| Used for | DFS, undo, backtracking | BFS, scheduling, buffers |

---

## Quick Check

```python
from collections import deque

q = deque()
q.append(10)
q.append(20)
q.append(30)
q.popleft()
q.append(40)
print(list(q))   # what does this print?
```

Then fire Antigravity:

```
Use skill `scaffold-code-exercise` to create a concept note and paired `.py` file for **Queue** in Python. Cover: FIFO concept vs LIFO stack, three implementation options (deque/list/Queue) with why deque wins, enqueue/dequeue/peek operations, complexity table, BFS pattern with adjacency list, level-order traversal pattern, and Stack vs Queue comparison table. Difficulty: beginner-intermediate. Tag it `python`, `data-structures`, `queue`, `collections`. Link to Python index, Stacks, and Deque notes.
```

In [16]:

# ─────────────────────────────────────────────────────────────────────
# LEETCODE #933 — Number of Recent Calls
# Difficulty: Easy
# ─────────────────────────────────────────────────────────────────────
#
# You are counting how many requests arrived in the last 3000ms.
#
# Write a class RecentCounter that:
#   - Starts with an empty record of requests
#   - Has a method ping(t) that:
#       * Records a new request at time t
#       * Returns how many requests happened in the range [t-3000, t]
#
# Each call to ping() is guaranteed to have a LARGER t than before.
#
# EXAMPLES:
#   ping(1)     → range [−2999,    1]  → 1 request in range  → return 1
#   ping(100)   → range [−2900,  100]  → 2 requests in range → return 2
#   ping(3001)  → range [1,     3001]  → 3 requests in range → return 3
#   ping(3002)  → range [2,     3002]  → 3 requests in range → return 3
#                                          (ping(1) fell out the back)
#
# THE QUEUE INSIGHT:
#   - Every ping goes into the BACK of the queue
#   - Any ping older than t-3000 is kicked off the FRONT
#   - Whatever is left in the queue is inside the window
#   - Queue length = answer
#
# CONSTRAINTS:
#   - 1 <= t <= 10^9
#   - Each call to ping has a strictly larger t than before
#   - At most 10^4 calls to ping
#
# TIME:  O(1) amortized — each request enters and leaves the queue once
# SPACE: O(n) — at most 3000 requests live in the queue at once
# ─────────────────────────────────────────────────────────────────────

from collections import deque


# ─────────────────────────────────────────────
#  Implementation
# ─────────────────────────────────────────────

class RecentCounter:

    def __init__(self):
        self.q = deque()          # holds timestamps of recent requests

    def ping(self, t: int) -> int:
        self.q.append(t)          # record the new request at the BACK

        # kick out anything older than the 3000ms window
        while self.q[0] < t - 3000:
            #print (self.q[0])
            self.q.popleft()      # too old — remove from FRONT

        return len(self.q)        # everything left is inside the window


# ─────────────────────────────────────────────
#  Test Harness
# ─────────────────────────────────────────────

def run_tests():

    tests = [
        # (pings,                      expected results,    description)
        ([1, 100, 3001, 3002],         [1, 2, 3, 3],        "LeetCode example"),
        ([1],                          [1],                  "single ping"),
        ([1, 3002],                    [1, 1],               "first ping falls out"),
        ([100, 200, 300, 400],         [1, 2, 3, 4],         "all within window"),
        ([1, 2, 3, 4000],              [1, 2, 3, 1],         "big jump — queue clears"),
    ]

    print(f"{'#':<4} {'Description':<30} {'Input':<40} {'Expected':<22} {'Got':<22} {'Pass?'}")
    print("─" * 82)

    for i, (pings, expected, desc) in enumerate(tests, 1):
        rc     = RecentCounter()          # fresh counter for each test
        result = [rc.ping(t) for t in pings]
        passed = "✅" if result == expected else "❌"
        print(f"{i:<4} {desc:<30} {str(pings):<40} {str(expected):<22} {str(result):<22} {passed}")


"""
Expected Output:
#    Description                    Input                                    Expected               Got                    Pass?
──────────────────────────────────────────────────────────────────────────────────
1    LeetCode example               [1, 100, 3001, 3002]                     [1, 2, 3, 3]           [1, 2, 3, 3]           ✅
2    single ping                    [1]                                      [1]                    [1]                    ✅
3    first ping falls out           [1, 3002]                                [1, 1]                 [1, 1]                 ✅
4    all within window              [100, 200, 300, 400]                     [1, 2, 3, 4]           [1, 2, 3, 4]           ✅
5    big jump — queue clears        [1, 2, 3, 4000]                          [1, 2, 3, 1]           [1, 2, 3, 1]           ✅
"""
run_tests()




#    Description                    Input                                    Expected               Got                    Pass?
──────────────────────────────────────────────────────────────────────────────────
1
1    LeetCode example               [1, 100, 3001, 3002]                     [1, 2, 3, 3]           [1, 2, 3, 3]           ✅
2    single ping                    [1]                                      [1]                    [1]                    ✅
1
3    first ping falls out           [1, 3002]                                [1, 1]                 [1, 1]                 ✅
4    all within window              [100, 200, 300, 400]                     [1, 2, 3, 4]           [1, 2, 3, 4]           ✅
1
2
3
5    big jump — queue clears        [1, 2, 3, 4000]                          [1, 2, 3, 1]           [1, 2, 3, 1]           ✅


In [18]:
# ─────────────────────────────────────────────────────────────────────
# LEETCODE #102 — Binary Tree Level Order Traversal  (BFS Pattern)
# Difficulty: Medium
# ─────────────────────────────────────────────────────────────────────
#
# Given a graph of nodes, visit every node level by level starting
# from a given start node. Return the order nodes were visited.
#
# WHAT IS BFS?
#   Visit all NEIGHBORS first before going deeper.
#   Think of it as ripples in a pond — closest nodes first.
#
# THE GRAPH:
#
#        A
#       / \
#      B   C
#       \ /
#        D
#
#   A knows B and C
#   B knows D
#   C knows D
#   D knows nobody
#
# WALKING THROUGH bfs(graph, "A"):
#
#   Start:    queue=[A]        visited={A}
#
#   Step 1:   pull A  → visit A  → add B, C to queue
#             queue=[B, C]     visited={A, B, C}
#
#   Step 2:   pull B  → visit B  → D not visited yet, add D
#             queue=[C, D]     visited={A, B, C, D}
#
#   Step 3:   pull C  → visit C  → D already visited, skip it
#             queue=[D]        visited={A, B, C, D}
#
#   Step 4:   pull D  → visit D  → no neighbors
#             queue=[]         done
#
#   Result:   [A, B, C, D]
#
# KEY TOOLS:
#   queue    = deque  — processes nodes in FIFO order (first in, first out)
#   visited  = set    — makes sure we never visit the same node twice
#
# TIME:  O(n)  — every node and edge visited once
# SPACE: O(n)  — queue and visited set hold at most n nodes
# ─────────────────────────────────────────────────────────────────────

from collections import deque


# ─────────────────────────────────────────────
#  Implementation
# ─────────────────────────────────────────────

def bfs(graph, start):
    queue   = deque([start])   # start with the first node in the queue
    visited = set([start])     # mark it visited immediately so we don't revisit
    order   = []               # records the order we visited nodes

    while queue:
        node = queue.popleft()          # grab the FRONT of the queue
        order.append(node)              # record the visit

        for neighbor in graph[node]:    # look at all neighbors
            if neighbor not in visited:
                visited.add(neighbor)   # mark before adding — prevents duplicates
                queue.append(neighbor)  # add to the BACK of the queue

    return order


# ─────────────────────────────────────────────
#  Test Harness
# ─────────────────────────────────────────────

def run_tests():

    tests = [
        # (graph,                                start,  expected,              description)
        ({"A": ["B","C"], "B": ["D"], "C": ["D"], "D": []},
                                                 "A",   ["A","B","C","D"],     "standard diamond shape"),

        ({"A": ["B"], "B": ["C"], "C": ["D"], "D": []},
                                                 "A",   ["A","B","C","D"],     "straight line A→B→C→D"),

        ({"A": ["B","C","D"], "B": [], "C": [], "D": []},
                                                 "A",   ["A","B","C","D"],     "star — A points to all"),

        ({"A": []},
                                                 "A",   ["A"],                 "single node no neighbors"),

        ({"A": ["B","C"], "B": ["C"], "C": []},
                                                 "A",   ["A","B","C"],         "C reachable two ways — visit once"),
    ]

    print(f"{'#':<4} {'Description':<35} {'Expected':<25} {'Got':<25} {'Pass?'}")
    print("─" * 95)

    for i, (graph, start, expected, desc) in enumerate(tests, 1):
        result = bfs(graph, start)
        passed = "✅" if result == expected else "❌"
        print(f"{i:<4} {desc:<35} {str(expected):<25} {str(result):<25} {passed}")

"""
Expected Output:
──────────────────────────────────────────────────────────────────────────────────────────────────
#    Description                         Expected                  Got                       Pass?
──────────────────────────────────────────────────────────────────────────────────────────────────
1    standard diamond shape              ['A', 'B', 'C', 'D']      ['A', 'B', 'C', 'D']      ✅
2    straight line A→B→C→D              ['A', 'B', 'C', 'D']      ['A', 'B', 'C', 'D']      ✅
3    star — A points to all             ['A', 'B', 'C', 'D']      ['A', 'B', 'C', 'D']      ✅
4    single node no neighbors           ['A']                      ['A']                      ✅
5    C reachable two ways — visit once  ['A', 'B', 'C']            ['A', 'B', 'C']            ✅
"""
run_tests()

#    Description                         Expected                  Got                       Pass?
───────────────────────────────────────────────────────────────────────────────────────────────
1    standard diamond shape              ['A', 'B', 'C', 'D']      ['A', 'B', 'C', 'D']      ✅
2    straight line A→B→C→D               ['A', 'B', 'C', 'D']      ['A', 'B', 'C', 'D']      ✅
3    star — A points to all              ['A', 'B', 'C', 'D']      ['A', 'B', 'C', 'D']      ✅
4    single node no neighbors            ['A']                     ['A']                     ✅
5    C reachable two ways — visit once   ['A', 'B', 'C']           ['A', 'B', 'C']           ✅


In [20]:
a = {"A": ["B","C"], "B": ["D"], "C": ["D"], "D": []}

it = iter(a)
while True:
    try:
        key = next(it)
        print(key, "->", a[key])
    except StopIteration:
        break

A -> ['B', 'C']
B -> ['D']
C -> ['D']
D -> []


In [28]:
# ─────────────────────────────────────────────────────────────────────
# LEETCODE #102 — Binary Tree Level Order Traversal
# Difficulty: Medium
# ─────────────────────────────────────────────────────────────────────
#
# Given the root of a binary tree, return the values of nodes
# grouped by level — left to right, top to bottom.
#
# THE TREE:
#
#            1           ← Level 0
#           / \
#          2   3         ← Level 1
#         / \   \
#        4   5   6       ← Level 2
#
# EXPECTED OUTPUT:
#   [[1], [2, 3], [4, 5, 6]]
#
# THE KEY TRICK — for _ in range(len(queue)):
#   Before processing, snapshot how many nodes are in the queue.
#   That count is exactly how many nodes belong to the CURRENT level.
#   Process only that many — everything added DURING the loop
#   belongs to the NEXT level and will be handled next iteration.
#
# WALKING THROUGH THE TREE ABOVE:
#
#   Start:      queue=[1]
#
#   Iteration 1: len=1  → pull 1        level=[1]
#                         add 2, 3      queue=[2, 3]
#                         result=[[1]]
#
#   Iteration 2: len=2  → pull 2        level=[2]
#                         add 4, 5      queue=[3, 4, 5]
#                       → pull 3        level=[2, 3]
#                         add 6         queue=[4, 5, 6]
#                         result=[[1], [2, 3]]
#
#   Iteration 3: len=3  → pull 4        level=[4]         no children
#                       → pull 5        level=[4, 5]      no children
#                       → pull 6        level=[4, 5, 6]   no children
#                         queue=[]
#                         result=[[1], [2, 3], [4, 5, 6]]
#
# CONSTRAINTS:
#   - Number of nodes: 0 to 2000
#   - Node values: -1000 to 1000
#
# TIME:  O(n)  — every node visited exactly once
# SPACE: O(n)  — queue holds at most one full level at a time
# ─────────────────────────────────────────────────────────────────────
def do_nothing():
    print("   ")
from collections import deque


# ─────────────────────────────────────────────
#  Building Blocks
# ─────────────────────────────────────────────

class Node:
    def __init__(self, val):
        self.val   = val
        self.left  = None
        self.right = None


# ─────────────────────────────────────────────
#  Implementation
# ─────────────────────────────────────────────

def level_order(root):
    if not root:
        return []

    queue  = deque([root])
    result = []

    while queue:
        level     = []
        level_size = len(queue)          # snapshot — how many nodes THIS level has

        for _ in range(level_size):      # process exactly that many — no more
            node = queue.popleft()
            level.append(node.val)

            if node.left:                # add children for NEXT level
                queue.append(node.left)
            if node.right:
                queue.append(node.right)

        result.append(level)             # done with this level — save it

    return result


# ─────────────────────────────────────────────
#  Helper — build tree from a list
#  Values fill left to right, None = empty spot
# ─────────────────────────────────────────────

def build(values):
    r"""
    build([1, 2, 3, 4, 5, None, 6]) produces:

            1
           / \
          2   3
         / \   \
        4   5   6
    """
    if not values:
        return None

    root  = Node(values[0])
    queue = deque([root])
    i     = 1

    while queue and i < len(values):
        node = queue.popleft()

        if i < len(values) and values[i] is not None:
            node.left = Node(values[i])
            queue.append(node.left)
        i += 1

        if i < len(values) and values[i] is not None:
            node.right = Node(values[i])
            queue.append(node.right)
        i += 1

    return root


# ─────────────────────────────────────────────
#  Test Harness
# ─────────────────────────────────────────────

def run_tests():

    tests = [
        # (tree as list,                    expected,                    description)
        ([1, 2, 3, 4, 5, None, 6],         [[1], [2,3], [4,5,6]],       "standard 3-level tree"),
        ([1],                              [[1]],                        "single node"),
        ([1, 2, None],                     [[1], [2]],                   "left child only"),
        ([1, None, 3],                     [[1], [3]],                   "right child only"),
        ([1, 2, 3],                        [[1], [2,3]],                 "two levels, full"),
        ([],                               [],                           "empty tree"),
    ]

    print(f"{'#':<4} {'Description':<25} {'Expected':<30} {'Got':<30} {'Pass?'}")
    print("─" * 95)

    for i, (tree, expected, desc) in enumerate(tests, 1):
        root   = build(tree)
        result = level_order(root)
        passed = "✅" if result == expected else "❌"
        print(f"{i:<4} {desc:<25} {str(expected):<30} {str(result):<30} {passed}")


run_tests()

r"""
Expected Output:
───────────────────────────────────────────────────────────────────────────────────────────────────
#    Description               Expected                        Got                            Pass?
───────────────────────────────────────────────────────────────────────────────────────────────────
1    standard 3-level tree     [[1], [2, 3], [4, 5, 6]]        [[1], [2, 3], [4, 5, 6]]        ✅
2    single node               [[1]]                            [[1]]                           ✅
3    left child only           [[1], [2]]                       [[1], [2]]                      ✅
4    right child only          [[1], [3]]                       [[1], [3]]                      ✅
5    two levels, full          [[1], [2, 3]]                    [[1], [2, 3]]                   ✅
6    empty tree                []                               []                              ✅
"""


do_nothing()

#    Description               Expected                       Got                            Pass?
───────────────────────────────────────────────────────────────────────────────────────────────
1    standard 3-level tree     [[1], [2, 3], [4, 5, 6]]       [[1], [2, 3], [4, 5, 6]]       ✅
2    single node               [[1]]                          [[1]]                          ✅
3    left child only           [[1], [2]]                     [[1], [2]]                     ✅
4    right child only          [[1], [3]]                     [[1], [3]]                     ✅
5    two levels, full          [[1], [2, 3]]                  [[1], [2, 3]]                  ✅
6    empty tree                []                             []                             ✅
   
